TODO

- solve the issue of invoking pkl too many times
- parametrize the "overlapping interval", from 1 second to a bit more
   

## Object criticality for better and safer navigation - GOAL 3

This notebook is the iterative version, to compute GOAL3 on multiple configurations.


Can be executed straighforwardly, after setting some paths. But to really understand what it does, the non-iterative version is recommended. It has many step-by-step comments.

In [1]:
import numpy as np
import json
import os
import os.path
import itertools
import unittest
import sklearn
import tqdm
import json, os
from glob import glob
import pandas
import math
import json
from operator import itemgetter
from typing import Callable
from nuscenes import NuScenes
from nuscenes.eval.prediction.splits import *
import nuscenes.eval.detection.config as cnfig
from nuscenes.eval.detection.configs import *
from nuscenes.eval.detection.data_classes import DetectionBox 
from nuscenes.eval.detection import *
import nuscenes.eval.detection.algo as ag
from nuscenes.eval.detection.data_classes import DetectionMetricData, DetectionConfig, DetectionMetrics, DetectionBox, \
    DetectionMetricDataList
from nuscenes.eval.common.data_classes import EvalBoxes
from typing import List, Dict, Callable, Tuple
from nuscenes.eval.common.utils import center_distance, scale_iou, yaw_diff, velocity_l2, attr_acc, cummean
import nuscenes.eval.detection.evaluate as dcl    
from nuscenes.prediction import *
from nuscenes.map_expansion.map_api import *
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from pkl.models import compile_model
from PIL import Image

import torch
import operator

%load_ext autoreload
%autoreload 2

In [2]:
def samp2mapname(samp, nusc):
    scene = nusc.get('scene', samp['scene_token'])
    log = nusc.get('log', scene['log_token'])
    return log['location']


In [3]:
def get_grid(point_cloud_range, voxel_size):
    lower = np.array(point_cloud_range[:(len(point_cloud_range) // 2)])
    upper = np.array(point_cloud_range[(len(point_cloud_range) // 2):])

    dx = np.array(voxel_size)
    bx = lower + dx/2.0
    nx = ((upper - lower) / dx).astype(int)

    return dx, bx, nx

In [4]:
#returns the trajectory as computed by pkl, in number_of_trajectory_poses frames.
#default is 16 frames (one each 0.25 seconds, for a total of 3 seconds)
#TRAJ_POINTS is the number of predicted trajectory positions (trajectory points) for each frame
def computetrajectory(sT, number_of_trajectory_poses=16, TRAJ_POINTS=25):
    dimension=sT[0][0].shape
    basic_shape=np.zeros(dimension)    
    trajectory=basic_shape
    for i in range(0,number_of_trajectory_poses):
        #get 20 most relevant points in sT[i][0].cpu().detach().numpy(), i.e., per frame
        tmp=sT[i][0].cpu().detach().numpy().copy()
        tmp_most_relevant_index=np.argsort(tmp.flatten())[-TRAJ_POINTS:]
        tmp.fill(0)
        flat=tmp.flatten()
        flat[tmp_most_relevant_index]=1
        tmp=np.reshape(flat, dimension)
        trajectory=np.add(trajectory,tmp)
        trajectory[trajectory>0]=1

    return trajectory


#as computetrajectory, but returns the trajectory between 2 of the trajectory frames
def computetrajectory_min_max(sT, minimum=0, maximum=16, number_of_trajectory_poses=16, TRAJ_POINTS=25):
    dimension=sT[0][0].cpu().detach().numpy().shape
    basic_shape=np.zeros(dimension)    
    trajectory=basic_shape
    for i in range(minimum,maximum):
        #get 20 most relevant points in sT[i][0].cpu().detach().numpy(), i.e., per frame
        tmp=sT[i][0].cpu().detach().numpy().copy()
        tmp_most_relevant_index=np.argsort(tmp.flatten())[-TRAJ_POINTS:]
        tmp.fill(0)
        flat=tmp.flatten()
        flat[tmp_most_relevant_index]=1
        tmp=np.reshape(flat, dimension)
        trajectory=np.add(trajectory,tmp)
        trajectory[trajectory>0]=1

    return trajectory

In [5]:
#from 1 layer image to RGB all black 
def toblack(image):
    black_image=np.ones((image.shape[0],image.shape[1],3))
    black_image[:,:,0][image>0]=0
    black_image[:,:,1][image>0]=0
    black_image[:,:,2][image>0]=0
    return black_image

def tored(image):
    red_image=np.ones((image.shape[0],image.shape[1],3))
#    red_image[:,:,0][image==0]=1
    red_image[:,:,1]=red_image[:,:,1]-image
    red_image[:,:,2]=red_image[:,:,1]-image
    return red_image
    
def toblu(image):
    blu_image=np.ones((image.shape[0],image.shape[1],3))
    blu_image[:,:,0]=blu_image[:,:,0]-image
    blu_image[:,:,1]=blu_image[:,:,1]-image
#    blu_image[:,:,2][image==0]=1
    return blu_image

    
def printfigure3x2(font, t_gt, t_pt, t_dt, gt, original_dt,dynamic_dt, ego):
    image_gt=np.ones((original_dt.shape[0],original_dt.shape[1],3))
    image_pt=np.ones((original_dt.shape[0],original_dt.shape[1],3))
    image_dt=np.ones((original_dt.shape[0],original_dt.shape[1],3))
    
    ego=tored(ego)
    t_gt=toblack(t_gt)
    t_pt=toblack(t_pt)
    t_dt=toblack(t_dt)

    gt=toblu(gt)
    original_dt=toblu(original_dt)
    dynamic_dt=toblu(dynamic_dt)
    plt.figure()
    
    fig, (ax) = plt.subplots(2,3)
    ax[0,0].tick_params(axis='both', labelsize=font)
    ax[0,1].tick_params(axis='both', labelsize=font)
    ax[0,2].tick_params(axis='both', labelsize=font)
    ax[1,0].tick_params(axis='both', labelsize=font)
    ax[1,1].tick_params(axis='both', labelsize=font)
    ax[1,2].tick_params(axis='both', labelsize=font)
    fig.tight_layout()

    ax[0,0].set_title('GT Traj', fontsize=font)
    ax[0,0].imshow(np.swapaxes(t_gt, 0, 1))

    ax[0,1].set_title('Original Traj', fontsize=font)
    ax[0,1].imshow(np.swapaxes(t_pt, 0, 1))

    ax[0,2].set_title('Dynamic Traj', fontsize=font)
    ax[0,2].imshow(np.swapaxes(t_dt, 0, 1))

    ax[1,0].set_title('GT BBox and GT Traj', fontsize=font)
    image_gt[t_gt==0]=0
    image_gt[gt==0]=0
    image_gt[ego==0]=0
    image_gt[image_gt>1]=1
    ax[1,0].imshow(np.swapaxes(image_gt, 0, 1))

    ax[1,1].set_title('Orig BBox and orig pkl', fontsize=font)
    image_pt[t_pt==0]=0
    image_pt[original_dt==0]=0
    image_pt[ego==0]=0
    image_pt[image_pt>1]=1
    ax[1,1].imshow(np.swapaxes(image_pt, 0, 1))

    ax[1,2].set_title('Dyn BBox and dyn pkl', fontsize=font)
    image_dt[t_dt==0]=0
    image_dt[dynamic_dt==0]=0
    image_dt[ego==0]=0
    image_dt[image_dt>1]=1
    ax[1,2].imshow(np.swapaxes(image_dt, 0, 1))
    plt.show()


def printfigure3x1(font, overlap, gt_with_respect_to_ego, \
                   trajectory_next_second, resulting_image):
    plt.clf()
    plt.figure()
    fig, (ax) = plt.subplots(1,3)
    ax[0].tick_params(axis='both', labelsize=font)
    ax[1].tick_params(axis='both', labelsize=font)
    ax[2].tick_params(axis='both', labelsize=font)
    fig.tight_layout()
        
    ax[0].set_title('GT BBs', fontsize=font)
    ax[0].imshow(np.swapaxes(gt_with_respect_to_ego, 0, 1))

    ax[1].set_title('Predict Traj', fontsize=font)
    ax[1].imshow(np.swapaxes(trajectory_next_second, 0, 1))

    ax[2].set_title('overlap: {}'.format(overlap), fontsize=font)
    ax[2].imshow(np.swapaxes(resulting_image, 0, 1))
    plt.show()    

In [6]:
def get_local_map(nmap, center, stretch, layer_names, line_names):
    # need to get the map here...
    box_coords = (
        center[0] - stretch,
        center[1] - stretch,
        center[0] + stretch,
        center[1] + stretch,
    )

    polys = {}

    # polygons
    records_in_patch = nmap.get_records_in_patch(box_coords,
                                                 layer_names=layer_names,
                                                 mode='intersect')
    for layer_name in layer_names:
        polys[layer_name] = []
        for token in records_in_patch[layer_name]:
            poly_record = nmap.get(layer_name, token)
            if layer_name == 'drivable_area':
                polygon_tokens = poly_record['polygon_tokens']
            else:
                polygon_tokens = [poly_record['polygon_token']]

            for polygon_token in polygon_tokens:
                polygon = nmap.extract_polygon(polygon_token)
                polys[layer_name].append(np.array(polygon.exterior.xy).T)

    # lines
    for layer_name in line_names:
        polys[layer_name] = []
        for record in getattr(nmap, layer_name):
            token = record['token']

            line = nmap.extract_line(record['line_token'])
            if line.is_empty:  # Skip lines without nodes
                continue
            xs, ys = line.xy

            polys[layer_name].append(
                np.array([xs, ys]).T
                )

    # convert to local coordinates in place
    rot = get_rot(np.arctan2(center[3], center[2])).T
    for layer_name in polys:
        for rowi in range(len(polys[layer_name])):
            polys[layer_name][rowi] -= center[:2]
            polys[layer_name][rowi] = np.dot(polys[layer_name][rowi], rot)

    return polys

In [7]:
#get the samples that follows my_selected_sample, for the defined seconds
def get_following_tokens(seconds, initial_token, sample):
    to_retrieve=seconds*2
    sequence_tokens=[] #list of tokens in sequences

    sequence_tokens.append(initial_token)
    for i in range(0, to_retrieve):
        following_token=sample['next']
        sequence_tokens.append(following_token)
        sample = nuscenes.get('sample', following_token)
    return sequence_tokens

#computes and overlaps all the bboxes
#list of tokens and info complete as from the createdimages_original
def getbboxes(tokens, info):
    return None


In [8]:
def samp2ego(samp, nusc):
    egopose = nusc.get('ego_pose', nusc.get('sample_data',
                                            samp['data']['LIDAR_TOP'])
                       ['ego_pose_token'])
    rot = Quaternion(egopose['rotation']).rotation_matrix
    rot = np.arctan2(rot[1, 0], rot[0, 0])
    return {
                'x': egopose['translation'][0],
                'y': egopose['translation'][1],
                'hcos': np.cos(rot),
                'hsin': np.sin(rot),
                'l': 4.084,
                'w': 1.73,
            }

#example output:
#samp2ego(nusc.get('sample', listtoken[1]), nusc)

#{'x': 1663.7830538752974,
# 'y': 1254.2029068912966,
# 'hcos': 0.4754610435577135,
# 'hsin': 0.8797367765752492,
# 'l': 4.084, #ego dimensions
# 'w': 1.73} #ego dimensions


In [9]:
#return the sum of all ground truth positions in the considered tokens, with respect to ego
def collect_x_gt(sample_tokens, #here put all the X tokens that we need to consider,
                 #the token with ego is the first token
              ego,            
              nusc,
              gt_boxes,
              nusc_maps,
              stretch=120,
              layer_names=['road_segment', 'lane'],
              line_names=['road_divider', 'lane_divider'],
              nx=256, ny=256, 
              bx=np.array([-16.85, -38.35]),
              dx=np.array([0.3, 0.3])):
    
    samp = nusc.get('sample', sample_tokens[0])
    # local map
    map_name = samp2mapname(samp, nusc)
    lmap = get_local_map(nusc_maps[map_name],
                             [ego['x'], ego['y'], ego['hcos'], ego['hsin']],
                             stretch, layer_names, line_names)    
    
    gtlobjs=np.empty((10000,4))
    gtlws=np.empty((10000, 2))
    counter=0
    for i in sample_tokens:
        sample_token=i
        samp = nusc.get('sample', sample_token)

        # ground truths bboxes
        gtlobjs_tmp, gtlws_tmp = get_other_objs(gt_boxes[sample_token],
                                        np.array([ego['x'], ego['y'],
                                                  ego['hcos'], ego['hsin']]))
        
        length=gtlobjs_tmp.shape[0]
        
        gtlobjs[counter:(counter+length)]=gtlobjs_tmp
        gtlws[counter:(counter+length)]=gtlws_tmp
        counter=counter+length

    gtlobjs=gtlobjs[0:counter]
    gtlws=gtlws[0:counter]
    # render
    gtx = raster_render(lmap, [ego['l'], ego['w']], gtlobjs, gtlws,
                            nx, ny, layer_names, line_names,
                            bx, dx)

    return gtx

#true if there are overlaps between the black area in a and b
#very simple:black zones becomes equal to 10
#if there is a 20, it means there is an overlap between images
def compute_overlaps(a, b):
    image=np.add(a, b)
    if(np.max(np.unique(image))==2):
        return True, image

    return False, image


#a = sum of ground truth
#b = trajectory computed 
def save_results(a, b, token, string):
    a[a==1]=0.5
    image=np.add(a, b)#ground truth light grey, trajectory in black
    image[image>1]=1
    im = Image.fromarray(image)
    if (im.mode != 'RGB'):
        im = im.convert('RGB')
    im.save(RESULTS_PATH+"/"+selected_detector+"/"+str(token)+"_"+str(string)+".jpeg")


In [10]:
#tokens is the list of "seconds*2" consecutive frames
#(should be 8 frames from nuscenes)
#computes overlaps in 1-seconds slots
def do_computation(tokens,
                   info_original,
                   all_original,
                   info_dynamic,
                   all_dynamic,
                   dx, bx, nx, ny, stretch, nusc, nusc_maps, gt_boxes
                  ):
    
    #get ego
    samp = nusc.get('sample', tokens[0])
    ego = samp2ego(samp, nusc)

    #get pkl score of the first token
    pkl_measured_original=info_original['full'][tokens[0]]
    pkl_measured_dynamic=info_dynamic['full'][tokens[0]]

    
    #get the ground truth trajectory
    sGT=all_original[0]['heat_gt']
    trajectory_pt=computetrajectory(sGT, number_of_trajectory_poses, TRAJ_POINTS)
    
    #get trajectory original
    sPT=all_original[0]['heat_pt']
    trajectory_pt=computetrajectory(sPT, number_of_trajectory_poses, TRAJ_POINTS)

    #get trajectory dynamic
    sDT=all_dynamic[0]['heat_pt']
    trajectory_pt=computetrajectory(sDT, number_of_trajectory_poses, TRAJ_POINTS)
    
    #get the ground_truths bboxes of each token, with respect to ego, and overlap trajectories
    for i in range(0, 6):#frames of nuscenes tokens list
        j=2*i #the 16 frames of the trajectory planner
        gt_with_respect_to_ego=collect_x_gt(tokens[i:i+3], #1 seconds
                                            ego, nusc, gt_boxes, nusc_maps,
                                            stretch=stretch, 
                                            layer_names=layer_names,
                                            line_names=line_names,
                                            nx=nx, ny=ny, bx=bx, dx=dx)[3]
    
        #gt trajectory and gt bboxes
        trajectory_next_second=computetrajectory_min_max(sGT, minimum=j,
                                                         maximum=j+5, #2 seconds
                                                         number_of_trajectory_poses=16,
                                                         TRAJ_POINTS=25)

        overlap_gt, resulting_image=compute_overlaps(gt_with_respect_to_ego.copy(),
                                                        trajectory_next_second.copy())
    
        if(overlap_gt==True):
            print("overlap is "+str(overlap_gt)+" for token "+str(tokens[0])+ " at iteration i="+str(i))
            save_results(gt_with_respect_to_ego.copy(), trajectory_next_second.copy(), token, "GT BBoxes and GT trajectory")

        printfigure3x1(font, overlap_gt, gt_with_respect_to_ego,trajectory_next_second, resulting_image)

        #original trajectory and gt bboxes
        trajectory_next_second=computetrajectory_min_max(sPT, minimum=j,
                                                         maximum=j+5, #2 seconds
                                                         number_of_trajectory_poses=16,
                                                         TRAJ_POINTS=25)

        overlap_pt, resulting_image=compute_overlaps(gt_with_respect_to_ego.copy(),
                                                        trajectory_next_second.copy())
    
        if(overlap_pt==True):
            print("overlap is "+str(overlap_pt)+" for token "+str(tokens[0])+ " at iteration i="+str(i))
            save_results(gt_with_respect_to_ego.copy(), trajectory_next_second.copy(), token, "GT BBoxes and original PT trajectory")

        printfigure3x1(font, overlap_pt, gt_with_respect_to_ego,trajectory_next_second, resulting_image)

            
        #dynamic trajectory and gt bboxes
        trajectory_next_second=computetrajectory_min_max(sDT, minimum=j,
                                                         maximum=j+5, #2 seconds
                                                         number_of_trajectory_poses=16,
                                                         TRAJ_POINTS=25)

        overlap_dt, resulting_image=compute_overlaps(gt_with_respect_to_ego.copy(),
                                                        trajectory_next_second.copy())
    
        if(overlap_dt==True):
            print("overlap is "+str(overlap_pt)+" for token "+str(tokens[0])+ " at iteration i="+str(i))
            save_results(gt_with_respect_to_ego.copy(), trajectory_next_second.copy(), token, "GT BBoxes and dynamic DT trajectory")

        printfigure3x1(font, overlap_dt, gt_with_respect_to_ego,trajectory_next_second, resulting_image)


        

In [11]:
def get_rot(h):
    return np.array([
        [np.cos(h), np.sin(h)],
        [-np.sin(h), np.cos(h)],
    ])


In [12]:
def objects2frame(history, center, toworld=False):
    """A sphagetti function that converts from global
    coordinates to "center" coordinates or the inverse.
    It has no for loops but works on batchs.
    """
    N, A, B = history.shape
    theta = np.arctan2(center[3], center[2])
    if not toworld:
        newloc = history[:, :, :2] - center[:2].reshape((1, 1, 2))
        rot = get_rot(theta).T
        newh = np.arctan2(history[:, :, 3], history[:, :, 2]) - theta
        newloc = np.dot(newloc.reshape((N*A, 2)), rot).reshape((N, A, 2))
    else:
        rot = get_rot(theta)
        newh = np.arctan2(history[:, :, 3], history[:, :, 2]) + theta
        newloc = np.dot(history[:, :, :2].reshape((N*A, 2)),
                        rot).reshape((N, A, 2))
    newh = np.stack((np.cos(newh), np.sin(newh)), 2)
    if toworld:
        newloc += center[:2]
    return np.append(newloc, newh, axis=2)


In [13]:
def raster_render(lmap, centerlw, lobjs, lws,
                  nx, ny, layer_names, line_names,
                  bx, dx):
    # draw both road layers vin one channel
    road_img = np.zeros((nx, ny))
    for layer_name in layer_names:
        for poly in lmap[layer_name]:
            # draw the lines
            pts = np.round(
                (poly - bx[:2] + dx[:2]/2.) / dx[:2]
            ).astype(np.int32)
            pts[:, [1, 0]] = pts[:, [0, 1]]
            cv2.fillPoly(road_img, [pts], 1.0)

    def draw_lane(layer_name, img):
        for poly in lmap[layer_name]:
            pts = np.round(
                (poly - bx[:2] + dx[:2]/2.) / dx[:2]
            ).astype(np.int32)
            pts[:, [1, 0]] = pts[:, [0, 1]]
            cv2.polylines(img, [pts], isClosed=False, color=1.0)
        return img
    road_div_img = np.zeros((nx, ny))
    draw_lane('road_divider', road_div_img)
    lane_div_img = np.zeros((nx, ny))
    draw_lane('lane_divider', lane_div_img)

    obj_img = np.zeros((nx, ny))
    for box, lw in zip(lobjs, lws):
        pts = get_corners(box, lw)
        # draw the box
        pts = np.round(
            (pts - bx[:2] + dx[:2]/2.) / dx[:2]
        ).astype(np.int32)
        pts[:, [1, 0]] = pts[:, [0, 1]]
        cv2.fillPoly(obj_img, [pts], 1.0)

    center_img = np.zeros((nx, ny))
    pts = get_corners([0.0, 0.0, 1.0, 0.0], centerlw)
    pts = np.round(
        (pts - bx[:2] + dx[:2]/2.) / dx[:2]
    ).astype(np.int32)
    pts[:, [1, 0]] = pts[:, [0, 1]]
    cv2.fillPoly(center_img, [pts], 1.0)

    return np.stack([road_img, road_div_img, lane_div_img,
                     obj_img, center_img])

In [14]:
def get_other_objs(boxes, ego):
    objs = []
    lws = []
    for box in boxes:
        rot = Quaternion(box.rotation).rotation_matrix
        rot = np.arctan2(rot[1, 0], rot[0, 0])

        lws.append([box.size[1], box.size[0]])
        objs.append([box.translation[0], box.translation[1],
                     np.cos(rot), np.sin(rot)])
    objs = np.array(objs)
    lws = np.array(lws)
    if len(objs) == 0:
        lobjs = np.zeros((0, 4))
    else:
        lobjs = objects2frame(objs[np.newaxis, :, :],
                              ego,
                              )[0]
    return lobjs, lws

#lws is the dimension of the bbox
#lobjs seems translation with respect to ego + rotation with respect to ego

In [15]:
def get_corners(box, lw):
    l, w = lw
    simple_box = np.array([
        [-l/2., -w/2.],
        [l/2., -w/2.],
        [l/2., w/2.],
        [-l/2., w/2.],
    ])
    h = np.arctan2(box[3], box[2])
    rot = get_rot(h)
    simple_box = np.dot(simple_box, rot)
    simple_box += box[:2]
    return simple_box

In [16]:
def create_dt(detector_file="none",
              val='val',
              model='none',
              d=1,
              r=1,
              t=1,
              verbose=False,
              recall_type="PRED AL NUMERATORE",
              nworkers=10, #for pkl
              bsz= 128, #for pkl
              gpuid=0# GPU ID, -1 if CPU only
             ):

    
    dt=dcl.DetectionEval(nusc=nuscenes,
        config=confvalue,
        result_path=detector_file,
        eval_set=val,
        model_name=model,
        MAX_DISTANCE_OBJ=d,
        MAX_DISTANCE_INTERSECT=r,
        MAX_TIME_INTERSECT_OBJ=t,
        verbose=verbose,
        recall_type="PRED AL NUMERATORE",
        nworkers=nworkers,
        bsz=bsz,
        gpuid=gpuid,output_dir='/home/notebook/pkl/results/GOAL3/remove/'
        )
    
    return dt

In [17]:
def compute_crit_pkl(dt,
                     listtoken=[],
                     conf_th_list=[0.4],
                     dist_list=[2.0],
                     crit_list=[0.5],
                     object_classes='car',
                     verbose=False):

    results= dt.safety_metric_evaluation(
                    list_of_tokens=listtoken,
                    conf_th_list=conf_th_list,
                    dist_list=dist,
                    crit_list=criticalities,
                    obj_classes_list=object_classes, 
                    render_images=False,
                    verbose=verbose
                    )
    
    return results

In [18]:
#computes if the target configuration leads to an improvement of pkl in some way
#fill results in a file containing:
#DETECTOR;
# pkl_dynamic['mean'], pkl_dynamic['median'], pkl_original['min']
#pkl_original['mean'],pkl_original['median'], pkl_original['max']
#pkl_GOAL1['mean'],pkl_GOAL1['median'], pkl_GOAL1['max']
#d, r, t
#criticality threshold for the dynamic
#confidence threshold for the dynamic
#threshold function for the dynamic
def compute_improvement(pkl_dynamic, pkl_original,pkl_GOAL1, detector, d, r, t, \
                        function_name, criticality, confidence):
    improvement=False
    mean_pkl_improvement=pkl_GOAL1['mean']-pkl_dynamic_results['mean']
    median_pkl_improvement=pkl_GOAL1['median']-pkl_dynamic_results['median']
    maximum_pkl_improvement=pkl_GOAL1['max']-pkl_dynamic_results['max']

    if(maximum_pkl_improvement>0 or mean_pkl_improvement > 0 or median_pkl_improvement> 0):
        improvement=True
        results={"detector": detector,
                "improvement":improvement,
                "function":str(function_name.__name__),
                "criticality":criticality,
                "confidence":confidence,
                "D, R, T":[d, r, t],
                "mean_pkl_improvement":mean_pkl_improvement,
                "median_pkl_improvement":median_pkl_improvement,
                "max_pkl_improvement":maximum_pkl_improvement,
                "mean_pkl_dynamic":pkl_dynamic['mean'],
                "median_pkl_dynamic":pkl_dynamic['median'],
                "max_pkl_dynamic":pkl_dynamic['max'],
                "mean_pkl_GOAL1":pkl_GOAL1['mean'],
                "median_pkl_GOAL1":pkl_GOAL1['median'],
                "max_pkl_GOAL1":pkl_GOAL1['max'],
                "mean_pkl":pkl_original['mean'],
                "median_pkl":pkl_original['median'],
                "max_pkl":pkl_original['max'],
           }

        #save results: first time, file does not exists
        if(os.path.isfile(RESULTS_PATH+detector+'/configurations_that_improve_pkl.json')==False):
            with open(RESULTS_PATH+detector+'/configurations_that_improve_pkl.json','w') as f:
                json.dump({detector: [results]}, f) 
        else: #file exists
            with open(RESULTS_PATH+detector+'/configurations_that_improve_pkl.json','r') as f:
                content=json.load(f) 
            with open(RESULTS_PATH+detector+'/configurations_that_improve_pkl.json','w+') as f:
                list_result=content[detector] #get the list of "results" stored in the file
                list_result.append(results)
                json.dump({detector:list_result}, f)


'''
def compute_improvement(pkl_dynamic, pkl_original, detector, d, r, t, \
                        function_name, criticality, confidence):
    improvement=False
    if(len(pkl_original)==1):
                      #pkl for the mean, median and max values
                      #computed using the best confidence threshold
                      #which is always the same, in this case
        pkl_original=[pkl_original[0],pkl_original[0],pkl_original[0]]
        
    mean_pkl_improvement=pkl_original[0]['mean']-pkl_dynamic_results['mean']
    median_pkl_improvement=pkl_original[1]['median']-pkl_dynamic_results['median']
    maximum_pkl_improvement=pkl_original[2]['max']-pkl_dynamic_results['max']

    if(maximum_pkl_improvement>0 or mean_pkl_improvement > 0 or median_pkl_improvement> 0):
        improvement=True
        results={"detector": detector,
                "improvement":improvement,
                "function":str(function_name.__name__),
                "criticality":criticality,
                "confidence":confidence,
                "D, R, T":[d, r, t],
                "mean_pkl_improvement":mean_pkl_improvement,
                "median_pkl_improvement":median_pkl_improvement,
                "max_pkl_improvement":maximum_pkl_improvement,
                "min_pkl_dynamic":pkl_dynamic['min'],
                "mean_pkl_dynamic":pkl_dynamic['mean'],
                "median_pkl_dynamic":pkl_dynamic['median'],
                "max_pkl_dynamic":pkl_dynamic['max'],
                "min_pkl_original":pkl_original[0]['min'],
                "mean_pkl_original":pkl_original[0]['mean'],
                "median_pkl_original":pkl_original[1]['median'],
                "max_pkl_original":pkl_original[2]['max'],
           }

        #save results: first time, file does not exists
        if(os.path.isfile(RESULTS_PATH+detector+'/configurations_that_improve_pkl.json')==False):
            with open(RESULTS_PATH+detector+'/configurations_that_improve_pkl.json','w') as f:
                json.dump({detector: [results]}, f) 
        else: #file exists
            with open(RESULTS_PATH+detector+'/configurations_that_improve_pkl.json','r') as f:
                content=json.load(f) 
            with open(RESULTS_PATH+detector+'/configurations_that_improve_pkl.json','w+') as f:
                list_result=content[detector] #get the list of "results" stored in the file
                list_result.append(results)
                json.dump({detector:list_result}, f)
''';

**Configuration items**

Configuration parameters. Rather intuitive: 

* set paths

* for all the rest, leave unaltered what you do not understand

In [19]:
GOAL="GOAL3" #set GOAL3, no alternative
car_only=False #filters out all non-car objects


verbose=False #strongly recommended "False"
nworkers=10 # number of workers for dataloader: depends on your resources!
bsz= 128# batch size for dataloader: depends on your resources!
gpuid=0 #id of the gpu; -1 if no GPU

DETECTOR= {
#           "SECFPN": 'SECFPN',
#           "FCOS3D": 'FCOS3D',
#           "PGD": 'PGD',
#           "SSN": 'SSN',
           "PointPillars": 'POINTP',
#           "RegNetX-1.6gF-FPN":'REG',
        }

#these below depends on the detector, it comes from GOAL 1
#depending on the detector, we have different thresholds that maximise pkl        
#and different d, r, t
#and different criticality scores
#same cases for both car_only==True and False
def set_threshold(detector):
    if(detector=='FCOS3D'):
        conf_list=list(np.arange(0.05, 0.35, 0.1))
    elif(detector=='PGD'):
        conf_list=list(np.arange(0.05, 0.35, 0.1))
    elif(detector=='POINTP'):
        conf_list=list(np.arange(0.5, 0.75, 0.05))
    elif(detector=='REG'):
        conf_list=list(np.arange(0.4, 0.7, 0.1))
    elif(detector=='SECFPN'):
        conf_list=list(np.arange(0.4, 0.7, 0.1))
    elif(detector=='SSN'):
        conf_list=list(np.arange(0.2, 0.5, 0.1))
    return conf_list

def set_d_r_t():
    MAX_D=list(range(20, 45, 5))
    MAX_R=list(range(5, 20, 5))
    MAX_T=list(range(4, 12, 4))
    #distance_intersect_time combinations
    DRT = list(itertools.product(*[MAX_D, MAX_R, MAX_T]))
    return DRT

#it is the optimal threshold for our detector; a unique value
#we retrieve experimentally
def get_optimal_threshold(detector):
    if(car_only==False):
        if(detector=='FCOS3D'):
            return [0.15, 0.25, 0.25]
        elif(detector=='PGD'):
            return [0.05]
        elif(detector=='POINTP'):
            return [0.55]
        elif(detector=='REG'):
            return [0.55]
        elif(detector=='SECFPN'):
            return [0.50]
        elif(detector=='SSN'):
            return [0.25, 0.30, 0.30]
        
    if(car_only==True):
        print("get_optimal_threshold car_only TODO")
        return " get_optimal_PKL car_only TODO"
    return "error"

def get_optimal_PKL(detector):
    if(car_only==False):
        if(detector=='FCOS3D'):
            combo= [4.104, 0.986, 98.922]
        elif(detector=='PGD'):
            combo= [4.778, 1.029, 116,456]
        elif(detector=='POINTP'):
            combo= [78.328, 24.857, 379.112]
        elif(detector=='REG'):
            combo= [78.951, 22.749, 380.456]
        elif(detector=='SECFPN'):
            combo= [65.561, 16.495, 390.986]
        elif(detector=='SSN'):
            combo= [3.490, 0.706, 118.750]
        
        return {"mean": combo[0], "median": combo[1], "max": combo[2]}
        
    if(car_only==True):
        print(" get_optimal_PKL car_only TODO")
        return " get_optimal_PKL car_only TODO"

def get_optimal_PKLGOAL1(detector):
    if(car_only==False):
        if(detector=='FCOS3D'):
            combo= [4.080,0.966, 98.922]
        elif(detector=='PGD'):
            combo= [4.741,0.973, 116.456]
        elif(detector=='POINTP'):
            combo= [45.046,8.985, 381.499 ]
        elif(detector=='REG'):
            combo= [43.801, 9.044, 378.434]
        elif(detector=='SECFPN'):
            combo= [38.284,8.378, 376.604 ]
        elif(detector=='SSN'):
            combo= [3.709, 0.719, 118.750]
        
        return {"mean": combo[0], "median": combo[1], "max": combo[2]}

    if(car_only==True):
        print("get_optimal_PKLGOAL1 car_only TODO")
        return "get_optimal_PKLGOAL1 car_only TODO"
    return "error"
        
def criticality_scores():
    return list(np.arange(0.9, 0.99, 0.05))


HOME='/home/notebook/'
#nuscene dataset install folder
DATAROOT = HOME+'nuscene/data/'

#pkl path to planner.pt and masks_trainval.json
modelpath=HOME+'/pkl/Evaluation-Of-Safety-Oriented-Metrics-for-Object-Detectors/metrics_model/planner.pt'
mask_json=HOME+'pkl/Evaluation-Of-Safety-Oriented-Metrics-for-Object-Detectors/metrics_model/masks_trainval.json'

#results of the object detectors (result_nusc.json) are stored here, in subdirectories:
#PATH+DETECTOR+FILE_JSON
#e.g. '/home/notebook/pkl/result_objdet/PGD/results_nusc.json'
PATH=HOME+'/pkl/result_objdet/'
FILE_JSON='/results_nusc.json'

#results computed are stored here
RESULTS_PATH=HOME+'/pkl/results/GOAL3/'

#the classes we consider. We can create "set of objects". It accepts 2 sets of objects.
#however we have a criticality model only for cars -- so do not remove cars from the list
#TODO WE NEED CURRENTLY TO EXECUTE EVERYTHING TWICE; THE object_classes_reduced does nothing
object_classes=['car', 'truck', 'bus', 'trailer', 'construction_vehicle', 'pedestrian', 'motorcycle', 'bicycle', 'traffic_cone', 'barrier']    
object_classes_reduced=['car']

if car_only:
    object_classes=object_classes_reduced

In [20]:
nuscenes = NuScenes('v1.0-trainval', dataroot=DATAROOT)
confvalue=cnfig.config_factory("detection_cvpr_2019")

Loading NuScenes tables for version v1.0-trainval...
23 category,
8 attribute,
4 visibility,
64386 instance,
12 sensor,
10200 calibrated_sensor,
2631083 ego_pose,
68 log,
850 scene,
34149 sample,
2631083 sample_data,
1166187 sample_annotation,
4 map,
Done loading in 28.018 seconds.
Reverse indexing ...
Done reverse indexing in 7.3 seconds.


In [21]:
#number of scene that compose the val set
eval=val = ['scene-0013',
                'scene-0554',
                'scene-0771',
                'scene-0929',
                'scene-1070',
                'scene-1072',
                'scene-0798',
                'scene-0108',
                'scene-0519',
                'scene-0332',
               ]

In [22]:
#get the list of tokens in the val set (150 scenes), put it in a file to be used as input
scenes_list=[]
counter=0

for i in nuscenes.scene:
    name=i['name']
    if(name in val):
        counter=counter+1
        scenes_list.append(i)

validation_samples={}
for i in scenes_list:
    scene_name=i['name']
    sample_token_list=[]
    first_sample_token=i['first_sample_token']
    last_sample_token=i['last_sample_token']
    current_sample_token=first_sample_token
    sample_token_list.append(current_sample_token)
    if(sample_token_list[0]!=first_sample_token):
        print("error")
        break
    while(current_sample_token!=last_sample_token):
        sample=nuscenes.get('sample', current_sample_token)
        current_sample_token=sample['next']
        sample_token_list.append(current_sample_token)
    if(sample_token_list[len(sample_token_list)-1]!=last_sample_token):
        print("error")
        break
    
    validation_samples.update({scene_name:sample_token_list})

listtoken=[]
for i in validation_samples.keys():
    listtoken.extend(validation_samples[i])

alltoken=listtoken

**Here we define the functions to define the thresholds in a dynamic way**

Currently we have just a simple condition:

- *if confidence_score > basic_threshold * corr_factor_score*  --> keep bbox

- *if criticality > basic_criticality and confidence_score < basic_threshold /corr_factor_score* --> keep bbox


To add more alternatives, just add a new function below.

In [23]:
DRT=set_d_r_t()#D,R,T values to try
criticality_th=criticality_scores() #criticality thresholds to consider

In [24]:
#we can add here more function to operate as correction factor    
def create_list_corr_factors(detector, conf_th_list):
    
    correction_factor1_parameters=list(itertools.product(*[criticality_th, conf_th_list]))

    return [{"correction_function": correction_factor1_25, "parameters":correction_factor1_parameters},
            {"correction_function": correction_factor1_2, "parameters":correction_factor1_parameters},
            {"correction_function": correction_factor1_3, "parameters":correction_factor1_parameters},
            {"correction_function": correction_factor1_35, "parameters":correction_factor1_parameters},
            # {"correction_function": correction_factor1_1, "parameters":correction_factor1_parameters} ,
           # {"correction_function": correction_factor0_9, "parameters":correction_factor1_parameters} 
           ]


#here is the first function we try
#each function has a different correction_factor
def correction_factor1_5(box_criticality, box_detection_score, criticality_th, conf_th):
    corr_factor_score=1.5
    if(box_detection_score > conf_th*corr_factor_score):
        return True
    elif(box_detection_score > conf_th/corr_factor_score and box_criticality> criticality_th ):
        return True
    else:
        return False

def correction_factor1_25(box_criticality, box_detection_score, criticality_th, conf_th):
    corr_factor_score=1.25
    if(box_detection_score > conf_th*corr_factor_score):
        return True
    elif(box_detection_score > conf_th/corr_factor_score and box_criticality> criticality_th ):
        return True
    else:
        return False


def correction_factor1_2(box_criticality, box_detection_score, criticality_th, conf_th):
    corr_factor_score=1.2
    if(box_detection_score > conf_th*corr_factor_score):
        return True
    elif(box_detection_score > conf_th/corr_factor_score and box_criticality> criticality_th ):
        return True
    else:
        return False

def correction_factor1_3(box_criticality, box_detection_score, criticality_th, conf_th):
    corr_factor_score=1.3
    if(box_detection_score > conf_th*corr_factor_score):
        return True
    elif(box_detection_score > conf_th/corr_factor_score and box_criticality> criticality_th ):
        return True
    else:
        return False

def correction_factor1_35(box_criticality, box_detection_score, criticality_th, conf_th):
    corr_factor_score=1.35
    if(box_detection_score > conf_th*corr_factor_score):
        return True
    elif(box_detection_score > conf_th/corr_factor_score and box_criticality> criticality_th ):
        return True
    else:
        return False

def correction_factor1_1(box_criticality, box_detection_score, criticality_th, conf_th):
    corr_factor_score=1.1
    if(box_detection_score > conf_th*corr_factor_score):
        return True
    elif(box_detection_score > conf_th/corr_factor_score and box_criticality> criticality_th ):
        return True
    else:
        return False

def correction_factor0_85(box_criticality, box_detection_score, criticality_th, conf_th):
    corr_factor_score=0.85
    if(box_detection_score > conf_th*corr_factor_score):
        return True
    elif(box_detection_score > conf_th/corr_factor_score and box_criticality> criticality_th ):
        return True
    else:
        return False

In [25]:
for detector in DETECTOR:
    #create results folder
    if not os.path.exists(RESULTS_PATH+DETECTOR[detector]+"/"): 
        print("creating results folder {}".format(RESULTS_PATH+DETECTOR[detector]+"/"))
        os.makedirs(RESULTS_PATH+DETECTOR[detector], exist_ok = False) 

Iterate on all detector to find if there are some nice configurations that we may want to explore later on, i.e., some cases where using our approach pkl is reduced either in mean, median or max pkl.

Results are saved in the GOAL3 folders that are created.

Each value is computed against the optimal pkl setting either for mean, median and max.

In [ ]:
for detector in DETECTOR:
    conf_th_list=set_threshold(DETECTOR[detector])  #confidence thresholds to try

    list_of_correction_factors=create_list_corr_factors(DETECTOR[detector], conf_th_list)

    pkl_crit_results_store=[]
    detector_file=PATH+DETECTOR[detector]+FILE_JSON
    optimal_thresholds=get_optimal_threshold(DETECTOR[detector])
    optimal_PKL=get_optimal_PKL(DETECTOR[detector])
    optimal_PKLGOAL1=get_optimal_PKLGOAL1(DETECTOR[detector])
        
    for d, r, t in DRT:
        print("create dt for {} {} {} {}".format(detector, d, r, t))
        dt=create_dt(detector_file,
                         'val',
                         model=detector_file,
                         d=d,
                         r=r,
                         t=t,
                         verbose=verbose,
                         recall_type="PRED AL NUMERATORE")
        '''
        #NO_correction_factor; just the plain pkl
        if(len(optimal_thresholds)==1):
            pkl_original=dt.calc_sample_crit_GOAL3(listtoken=alltoken,
                                             conf_th=optimal_thresholds[0],
                                             crit=5.0,#not considered as it is the "normal" pkl
                                             correction_factor=None, #in this case, it is just the normal pkl
                                             obj_classes_list=object_classes,#filter boxes based on class
                                             verbose=verbose)
            pkl_original=[pkl_original]
            print("[pkl original: mean {}, median {}, max {}]".format(pkl_original[0]['mean'], pkl_original[0]['median'], pkl_original[0]['max']) )
            
        else:#len is 3, we have an optimal threshold that is different for mean, median and max values
            pkl_original_mean_best=dt.calc_sample_crit_GOAL3(listtoken=alltoken,
                                             conf_th=optimal_thresholds[0],#conf_th for the best mean pkl
                                             crit=5.0,#not considered as it is the "normal" pkl
                                             correction_factor=None, #in this case, it is just the normal pkl
                                             obj_classes_list=object_classes,#filter boxes based on class
                                             verbose=verbose)
            
            pkl_original_median_best=dt.calc_sample_crit_GOAL3(listtoken=alltoken,
                                             conf_th=optimal_thresholds[1],#conf_th for the best median pkl
                                             crit=5.0,#not considered as it is the "normal" pkl
                                             correction_factor=None, #in this case, it is just the normal pkl
                                             obj_classes_list=object_classes,#filter boxes based on class
                                             verbose=verbose)

            pkl_original_max_best=dt.calc_sample_crit_GOAL3(listtoken=alltoken,
                                             conf_th=optimal_thresholds[2],#conf_th for the best max pkl
                                             crit=5.0,#not considered as it is the "normal" pkl
                                             correction_factor=None, #in this case, it is just the normal pkl
                                             obj_classes_list=object_classes,#filter boxes based on class
                                             verbose=verbose)

            pkl_original=[pkl_original_mean_best, pkl_original_median_best, pkl_original_max_best]
            print("[pkl original: mean {}, median {}, max {}]".format(pkl_original_mean_best[0]['mean'], pkl_original_median_best[0]['median'], pkl_original_max_best[0]['max']) )
        ''';

        for function in list_of_correction_factors:#function to compute the multiple threshold
            parameters=function['parameters']
            correction_function=function['correction_function']
            for criticality, confidence in parameters:
                print("function {}, criticality {}, confidence {}, drt ({} {} {}) ".format(str(function['correction_function'].__name__), criticality, confidence, d,r,t))
                #correction function is now active
                pkl_dynamic_results=dt.calc_sample_crit_GOAL3(listtoken=alltoken,
                                                     conf_th=confidence,
                                                     crit=criticality,
                                                     correction_factor=correction_function,
                                                     obj_classes_list=object_classes,#filter boxes based on class
                                                     verbose=verbose)
                
                compute_improvement(pkl_dynamic_results, optimal_PKL,optimal_PKLGOAL1, DETECTOR[detector], d, r, t, correction_function, criticality,confidence)
                print("[pkl dynamic: mean {}, median {}, max {}]".format(pkl_dynamic_results['mean'], pkl_dynamic_results['median'],pkl_dynamic_results['max']) )


create dt for PointPillars 20 5 4


function correction_factor1_25, criticality 0.9, confidence 0.5, drt (20 5 4) 
loading pkl model


/home/notebook/anaconda3/envs/openmmlab/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/notebook/anaconda3/envs/openmmlab/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


pkls computed with output of size : 403
[pkl dynamic: mean 76.8910903930664, median 25.68929672241211, max 381.71044921875]
function correction_factor1_25, criticality 0.9, confidence 0.55, drt (20 5 4) 
loading pkl model
pkls computed with output of size : 403
[pkl dynamic: mean 72.39448547363281, median 23.092151641845703, max 386.60845947265625]
function correction_factor1_25, criticality 0.9, confidence 0.6000000000000001, drt (20 5 4) 
loading pkl model
pkls computed with output of size : 403
[pkl dynamic: mean 66.89276885986328, median 22.25596809387207, max 379.06182861328125]
function correction_factor1_25, criticality 0.9, confidence 0.6500000000000001, drt (20 5 4) 
loading pkl model
pkls computed with output of size : 403
[pkl dynamic: mean 64.39440155029297, median 22.408906936645508, max 384.04364013671875]
function correction_factor1_25, criticality 0.9, confidence 0.7000000000000002, drt (20 5 4) 
loading pkl model
pkls computed with output of size : 403
[pkl dynamic: me

Let's check if we have improvements, and what are the most relevant improvements. These are the nice configurations to maintain.

In [ ]:
path= "/home/notebook/pkl/results/GOAL3/"
FILENAME='configurations_that_improve_pkl.json'
            
for i in DETECTOR:
    print(DETECTOR[i])
    d1=get_optimal_PKLGOAL1(DETECTOR[i])
    mean_GOAL1, median_GOAL1, max_GOAL1=d1['mean'], d1['median'], d1['max']
    d2=get_optimal_PKL(DETECTOR[i])
    mean_PKL, median_PKL, max_PKL=d2['mean'], d2['median'], d2['max']
    
    if(os.path.isfile(RESULTS_PATH+DETECTOR[i]+'/configurations_that_improve_pkl.json')==True):
        with open(RESULTS_PATH+DETECTOR[i]+'/configurations_that_improve_pkl.json','r') as f:
            data=json.load(f)
    else:
        print("file not found for {}"+i)
        continue

    for d in data:
        best_mean_data=None
        best_median_data=None
        best_max_data=None
        mean_local=99999
        median_local=9999
        max_local=9999
        for j in data[d]:
            if(mean_local > j['mean_pkl_dynamic']):
                mean_local=j['mean_pkl_dynamic'] 
                best_mean_data=j
            if(median_local > j['median_pkl_dynamic']):
                median_local = j['median_pkl_dynamic']
                best_median_data=j
            if(max_local >  j['max_pkl_dynamic']):
                max_local=  j['max_pkl_dynamic']
                best_max_data=j

        if(mean_local<mean_GOAL1):
            print("mean better than GOAL1 and OriginalPKL for {}".format(i))
            print(best_mean_data)
        elif(mean_local<mean_PKL):
            print("mean better than OriginalPKL for {}".format(i))
            print(best_mean_data)
        if(median_local<median_GOAL1):
            print("median better than GOAL1 and OriginalPKL for {}".format(i))
            print(best_median_data)
        elif(median_local<median_PKL):
            print("median better than OriginalPKL for {}".format(i))
            print(best_mean_data)
        if(max_local<max_GOAL1):
            print("max better than GOAL1 and OriginalPKL for {}".format(i))
            print(best_max_data)
        elif(max_local<max_PKL):
            print("max better than OriginalPKL for {}".format(i))
            print(best_max_data)

**The step below does not need a for loop, so it is removed. It is maintained in the other notebook, if needed to create some nice images, and can be used to investigate specific settings.**

We select most interesting frames, i.e., those we have most relevant improvement, and we show the corresponding pictures.

In [ ]:
'''X=len(pkl_dynamic_results['full']) #total number of frames to analyze
X
''';

In [ ]:
'''#Convert dictionary to list

#pkl_d is the list of tokens for dynamic pkl (our modified function)
#pkl_o is the list of tokens for original pkl function
pkl_d, pkl_o=pkl_dynamic_results['full'], pkl_original['full']
list_pkl_d=[]
list_pkl_o=[]

for a in iter(pkl_d):
    list_pkl_d.append([a, pkl_d[a]])

for a in iter(pkl_o):
    list_pkl_o.append([a, pkl_o[a]])
''';

In [ ]:
'''only_pkl_values=[]
for i in list_pkl_d:
    only_pkl_values.append(i[1])

top_X_index=np.argsort(only_pkl_values)[-X:]
top_X_index_pkl_d=itemgetter(*top_X_index)(list_pkl_d)

only_pkl_values=[]
for i in list_pkl_o:
    only_pkl_values.append(i[1])

top_X_index=np.argsort(only_pkl_values)[-X:]
top_X_index_pkl_o=itemgetter(*top_X_index)(list_pkl_o)

print("we have sorted the {} tokens based on pkl, for dynamic and original".format(X))
''';

Select:
    
1- n frame with maximum difference between pkl with original function and our modified function (i.e., our modified function is closer to the ground truth)<br>
2- k frame where original function has maximum pkl<br>
3- k frame where dynamic function has maximum pkl<br>

if the frames are the same in points 1, 2, 3, only one sample is shown

In [ ]:
'''
n=5
k=5
''';

In [ ]:
'''
#k frame where original function has maximum pkl
original_max=top_X_index_pkl_o[-k:]
#k frame where dynamic function has maximum pkl
dynamic_max=top_X_index_pkl_d[-k:]

#n frame of pkl difference maximum between original and dynamic (dynamic is better)
tmp=[]
for i in range(0, len(list_pkl_d)):
    value=list_pkl_o[i][1]-list_pkl_d[i][1]
    if(value>0):
        tmp.append([list_pkl_d[i][0], value])
max_gain_d_over_o=sorted(tmp, key=operator.itemgetter(1))[-n:]


#additional information
np_pkl_o=np.asarray(list_pkl_o)[:,1].astype(float)
np_pkl_d=np.asarray(list_pkl_d)[:,1].astype(float)
difference=np_pkl_d-np_pkl_o

print("number of values where dynamic pkls is equal than original is {}".format(len(difference[difference==0])))

#difference: dynamic smaller than original
print("number of values where dynamic pkls are smaller than original is {}".format(len(difference[difference<0])))

#index sorted, from lowest (most different) to highest"
dynamic_smaller_index=np.argsort(difference)
top_X_pkl_d_smaller=itemgetter(*dynamic_smaller_index[:X])(list_pkl_d)

#difference: dynamic bigger than original
print("number of values where dynamic pkls are larger than original is {}".format(len(difference[difference>0])))

#original_smaller_index=np.argsort(difference)
#top_X_pkl_o_smaller=itemgetter(*original_smaller_index[-X:])(list_pkl_o)
#print("indexes for dynamic smaller, and for original smaller are respectively in top_X_pkl_d_smaller, top_X_pkl_o_smaller ")
#print("dynamic: {}".format(top_X_pkl_d_smaller))
#print("original: {}".format(top_X_pkl_o_smaller)) 
''';

*and finally we visualize the results!*

We use a modified version of the pkl library and related functions, so that we can visualize the required information.
We obtain the following variables. We report them here as they are a too many and confusing.

- pklfile_original: only configuration parameters, e.g., the handler to the correction factor used
- pklfile_dynamic:  only configuration parameters, e.g., the handler to the correction factor used


- info_original: pkl min, max, mean, median computed on the target sample, using the traditional approach (no correction factor). If it is only 1 token, expect a unique value
- info_dynamic : pkl min, max, mean, median  computed on the target sample, using the correction factor. If it is only 1 token, expect a unique value


- all_pkls_original: pkl measures with no correction factor (original approach) 
- all_pkls_dynamic: pkl measures with correction factor \


- gtdist: ground truth trajectories. It is an array of shape [x, 16, 256, 256]. For each of the X images, there are 16 frames of shape [256, 256] frames: 16 planned trajectories through time, from t+0.25 until t+4.0 seconds. In other words, it is the forecast at time {t + 0.25k | 1 ≤ k ≤ 16}. The trajectory is plotted as "most likely trajectory" on a shape of [256, 256].
- preddist_original: like gtdist, but for trajectories planned using the predicted bbox (and original threshold)

- preddist_dynamic:  like gtdist, but for trajectories planned using the predicted bbox with the correction factor


- gtxs: array of shape [x, 5, 256, 256], where for each image X, it has:
    - position 0: the road
    - position 1: road separators, if any
    - position 2: lane separators, if any
    - position 3: ground truth position of objects
    - position 4: position of ego
    
- predxs_original: same as gtxs, but for prediction with no correction factor (original approach). The difference with respect to gtxs is only on position of objects.
- predxs_dynamic: same as gtxs, but for prediction with correction factor (our approach). The difference with respect to gtxs is only on position of objects.


- createdimages_original: a list, each element corresponds to a token (image). For each element, there is a dictionary. The dictionary contains: 
    - heat_gt: ground truth  trajectory, with 16 different trajectory poses, from t+0.25 until t+4.0 seconds. In practise, each of the 16 trajectory is conserved in a list of two elements: a tensor [256, 256] and an array [256,256,4]. For visualization, the array is better (just use showimg and a coloured picture appears), but the data is the same.
    - heat_pt:  predicted trajectory. 16 different trajectory poses, from t+0.25 until t+4.0 seconds. Structure is as above.
    - i: counter of the token (sample) processed
    - gtx: equivalent of gtxs for a single specific token
    - ptx: equivalent of predxs_original for a single specific token

- createdimages_dynamic: a list, each element corresponds to a token (image). For each element, there is a dictionary. The dictionary contains: (as above but for pkl with correction factor)
    - heat_gt: ground truth  trajectory. 16 different trajectory poses, from t+0.25 until t+4.0 seconds
    - heat_pt:  predicted trajectory. 16 different trajectory poses, from t+0.25 until t+4.0 seconds
    - i: counter of the token (sample) processed
    - gtx: equivalent of gtxs for a single specific token
    - ptx: equivalent of predxs_dynamic for a single specific token

Also, there is a saved image as jpg, but the information of the image is contained in the above values. Just ignore it.

**k frame where original function has maximum pkl**

In [ ]:
'''
number_of_trajectory_poses=16
TRAJ_POINTS=25
font=8
''';

In [ ]:
'''
images_to_show=[item[0] for item in original_max]

#first we consider worst cases with original pkl
pklfile_original, info_original, all_pkls_original, gtdist,preddist_original,gtxs, predxs_original, \
createdimages_original, gt_boxes,nusc_maps=dt.calc_sample_crit_GOAL3_visualization(listtoken=images_to_show,
                                     conf_th=basic_threshold,
                                     crit=basic_criticality,
                                     correction_factor=None, #in this case, it is just the normal pkl
                                     obj_classes_list=object_classes,#filter boxes based on class
                                     image_counter=len(images_to_show),
                                     verbose=verbose);

pklfile_dynamic, info_dynamic, all_pkls_dynamic, gtdist,preddist_dynamic,gtxs, predxs_dynamic, \
createdimages_dynamic, gt_boxes, nusc_maps=dt.calc_sample_crit_GOAL3_visualization(listtoken=images_to_show,
                                     conf_th=basic_threshold,
                                     crit=basic_criticality,
                                     correction_factor=analyzed_correction_factor, #in this case, it is just the normal pkl
                                     obj_classes_list=object_classes,#filter boxes based on class
                                     image_counter=len(images_to_show),
                                     verbose=verbose);
number_of_trajectory_poses=16
TRAJ_POINTS=25
font=8

%matplotlib inline
for token_original, token_dynamic in zip(createdimages_original,createdimages_dynamic):
    #get the ground truth bboxes
    sGT=token_original['heat_gt']
    ground_truths=token_original['gtx'].cpu().detach().numpy()[3]
    ego=token_original['gtx'].cpu().detach().numpy()[4]
    
    #get the ground truth trajectory
    trajectory_gt=computetrajectory(sGT.copy(), number_of_trajectory_poses, TRAJ_POINTS)
    
    #get the predicted original bboxes
    sPT=token_original['heat_pt']
    original_detections=token_original['ptx'].cpu().detach().numpy()[3]
    
    #get the predicted original trajectories
    trajectory_pt=computetrajectory(sPT.copy(), number_of_trajectory_poses, TRAJ_POINTS)
    
    #get the predicted dynamic (our approach) bboxes
    sDT=token_dynamic['heat_pt']
    dynamic_detections=token_dynamic['ptx'].cpu().detach().numpy()[3]

    #get the predicted dynamic (our approach) trajectories
    trajectory_dt=computetrajectory(sDT.copy(), number_of_trajectory_poses, TRAJ_POINTS)


    plt.clf()
    printfigure3x2(font,#font
                   trajectory_gt,#trajectory ground truth
                   trajectory_pt,#trajectory predicted by original pkl
                   trajectory_dt,#trajectory predicted by our approach to pkl
                   ground_truths,#ground truth position of objects
                   original_detections,#position of objects detected by original approach
                   dynamic_detections,#position of objects detected by our approach (threshold filtering)
                   ego
                  )

''':

**k frame where dynamic function has maximum pkl**

In [ ]:
'''
images_to_show=[item[0] for item in dynamic_max]

#first we consider worst cases with original pkl
pklfile_original, info_original, all_pkls_original, gtdist,preddist_original,gtxs, predxs_original, \
createdimages_original, gt_boxes, nusc_maps=dt.calc_sample_crit_GOAL3_visualization(listtoken=images_to_show,
                                     conf_th=basic_threshold,
                                     crit=basic_criticality,
                                     correction_factor=None, #in this case, it is just the normal pkl
                                     obj_classes_list=object_classes,#filter boxes based on class
                                     image_counter=len(images_to_show),
                                     verbose=verbose);

pklfile_dynamic, info_dynamic, all_pkls_dynamic, gtdist,preddist_dynamic,gtxs, predxs_dynamic, \
createdimages_dynamic, gt_boxes, nusc_maps=dt.calc_sample_crit_GOAL3_visualization(listtoken=images_to_show,
                                     conf_th=basic_threshold,
                                     crit=basic_criticality,
                                     correction_factor=analyzed_correction_factor, #in this case, it is just the normal pkl
                                     obj_classes_list=object_classes,#filter boxes based on class
                                     image_counter=len(images_to_show),
                                     verbose=verbose);

%matplotlib inline
for token_original, token_dynamic in zip(createdimages_original,createdimages_dynamic):
    #get the ground truth bboxes
    sGT=token_original['heat_gt']
    ground_truths=token_original['gtx'].cpu().detach().numpy()[3]
    ego=token_original['gtx'].cpu().detach().numpy()[4]
    #ground_truths=np.swapaxes(ground_truths,0,1)
    
    #get the ground truth trajectory
    trajectory_gt=computetrajectory(sGT.copy(), number_of_trajectory_poses, TRAJ_POINTS)
    #trajectory_gt=np.swapaxes(trajectory_gt, 0,1)   
    
    #get the predicted original bboxes
    sPT=token_original['heat_pt']
    original_detections=token_original['ptx'].cpu().detach().numpy()[3]
    #original_detections=np.swapaxes(original_detections,0,1)
    
    #get the predicted original trajectories
    trajectory_pt=computetrajectory(sPT.copy(), number_of_trajectory_poses, TRAJ_POINTS)
    #trajectory_pt=np.swapaxes(trajectory_pt, 0,1)
    
    #get the predicted dynamic (our approach) bboxes
    sDT=token_dynamic['heat_pt']
    dynamic_detections=token_dynamic['ptx'].cpu().detach().numpy()[3]
    #dynamic_detections=np.swapaxes(dynamic_detections,0,1)

    #get the predicted dynamic (our approach) trajectories
    trajectory_dt=computetrajectory(sDT.copy(), number_of_trajectory_poses, TRAJ_POINTS)
    #trajectory_dt=np.swapaxes(trajectory_dt, 0,1)


    plt.clf()
    printfigure3x2(font,#font
                   trajectory_gt,#trajectory ground truth
                   trajectory_pt,#trajectory predicted by original pkl
                   trajectory_dt,#trajectory predicted by our approach to pkl
                   ground_truths,#ground truth position of objects
                   original_detections,#position of objects detected by original approach
                   dynamic_detections,#position of objects detected by our approach (threshold filtering)
                   ego
                  )
''';

**n frame with maximum difference between pkl with original function and our modified function (i.e., our modified function is closer to the ground truth)**

In [ ]:
'''
images_to_show=[item[0] for item in max_gain_d_over_o]

#first we consider worst cases with original pkl
pklfile_original, info_original, all_pkls_original, gtdist,preddist_original,gtxs, predxs_original, \
createdimages_original, gt_boxes, nusc_maps=dt.calc_sample_crit_GOAL3_visualization(listtoken=images_to_show,
                                     conf_th=basic_threshold,
                                     crit=basic_criticality,
                                     correction_factor=None, #in this case, it is just the normal pkl
                                     obj_classes_list=object_classes,#filter boxes based on class
                                     image_counter=len(images_to_show),
                                     verbose=verbose);

pklfile_dynamic, info_dynamic, all_pkls_dynamic, gtdist,preddist_dynamic,gtxs, predxs_dynamic, \
createdimages_dynamic, gt_boxes, nusc_maps=dt.calc_sample_crit_GOAL3_visualization(listtoken=images_to_show,
                                     conf_th=basic_threshold,
                                     crit=basic_criticality,
                                     correction_factor=analyzed_correction_factor, #in this case, it is just the normal pkl
                                     obj_classes_list=object_classes,#filter boxes based on class
                                     image_counter=len(images_to_show),
                                     verbose=verbose);

%matplotlib inline
for token_original, token_dynamic in zip(createdimages_original, createdimages_dynamic):
    #get the ground truth bboxes
    sGT=token_original['heat_gt']
    ground_truths=token_original['gtx'].cpu().detach().numpy()[3]
    ego=token_original['gtx'].cpu().detach().numpy()[4]
    #ground_truths=np.swapaxes(ground_truths,0,1)
    
    #get the ground truth trajectory
    trajectory_gt=computetrajectory(sGT.copy(), number_of_trajectory_poses, TRAJ_POINTS)
    #trajectory_gt=np.swapaxes(trajectory_gt, 0,1)   
    
    #get the predicted original bboxes
    sPT=token_original['heat_pt']
    original_detections=token_original['ptx'].cpu().detach().numpy()[3]
    #original_detections=np.swapaxes(original_detections,0,1)
    
    #get the predicted original trajectories
    trajectory_pt=computetrajectory(sPT.copy(), number_of_trajectory_poses, TRAJ_POINTS)
    #trajectory_pt=np.swapaxes(trajectory_pt, 0,1)
    
    #get the predicted dynamic (our approach) bboxes
    sDT=token_dynamic['heat_pt']
    dynamic_detections=token_dynamic['ptx'].cpu().detach().numpy()[3]
    #dynamic_detections=np.swapaxes(dynamic_detections,0,1)

    #get the predicted dynamic (our approach) trajectories
    trajectory_dt=computetrajectory(sDT.copy(), number_of_trajectory_poses, TRAJ_POINTS)
    #trajectory_dt=np.swapaxes(trajectory_dt, 0,1)

    plt.clf()
    printfigure3x2(font,#font
                   trajectory_gt,#trajectory ground truth
                   trajectory_pt,#trajectory predicted by original pkl
                   trajectory_dt,#trajectory predicted by our approach to pkl
                   ground_truths,#ground truth position of objects
                   original_detections,#position of objects detected by original approach
                   dynamic_detections,#position of objects detected by our approach (threshold filtering)
                   ego
                  )

''';

**Understand if there are safety violations**

*We want to identify specific cases, to understand what would happen, and, especially, if we improve safety, reducing the risk of crashes.*

To proceed on this, we check if there are some "unsafe trajectory". We are inspired by the rule of the 3-second distance, according to the US National Authorities [1], [2]: this rule establishes that you are driving with a safe distance if it would take you 3 seconds to reach the position of the vehicle in front of you (i.e., supposed the vehicle in front freezes instantly, it would take 3 seconds to bump into it).

[1] Reference Material for DDC Instructors, 5th Edition. National Safety Council, 2005.

[2] https://www.travelers.com/resources/auto/travel/3-second-rule-for-safe-following-distance


Inspired by this, we consider the 4 seconds trajectory planned by pkl. We check the projected trajectories, and we see if the projected position of the ego vehicle in 4 seconds will lead to an overlap with the ground truth objects projected in the next 4 seconds.

For this, we first select the relevant frames. Realistically we should select frames which represent the most critical scenarios. However, it is difficult to say this from pkl, as it is mostly a reliability metric rather than a safety metric, as demonstrated in the SAC paper we published recently. So, pin-pointing a special frame (maybe the one which shows the best pkl improvement, or the one that mostly reduces the maximum pkl) is good only for visualization purposes.

We want to plot "up to 4 seconds": we do not apply on the 20 seconds length, but only on the first 12 seconds of each sequence.

We see if the projected position of the ego vehicle in 4 seconds will lead to an overlap with the ground truth objects.

- For the ground truth pkl
- the pkl using the bbox predicted with the original function.
- the pkl using the bbox predicted with our modified function.

In [ ]:
seconds=4

we first compute pkls for all val set considered

In [ ]:
len(alltoken)

we retrieve the nuscene scenes of interest

In [ ]:
#retrieve initial token of each nuscene scene in val
#Elements as:
#{'token': '41fde20fedcd4d22ab26811688612870',
#  'log_token': 'd31dc715d1c34b99bd5afb0e3aea26ed',
#  'nbr_samples': 40,
#  'first_sample_token': 'b9c60cfaf1814c8bb363037dec7abd35',
#  'last_sample_token': '51f7ea7d3e4e42dfaf183dbe9996f1fb',
#  'name': 'scene-0013',
#  'description': 'Follow bus, parked cars, trash cans, parked bicycles, nature'},

nuscenes_sequences=[]

for i in nuscenes.scene:
    if (i['name'] in val):
        nuscenes_sequences.append(i)    

for each token, we compute the successive 8 and we perform the "do_computation" that does essentially everything

In [ ]:
%matplotlib inline
#get the related sequence, to compute predictions number of images we have used in our validation
#first we get the my_sample with the reference selected_token
escape=0
dx,bx, (nx, ny) = get_grid([-17.0, -38.5, 60.0,38.5], [0.3, 0.3])
#dx,bx, (nx, ny) = get_grid([-34.0, -77, 120.0,77], [0.5, 0.5])
stretch=70 #120
layer_names = ['road_segment', 'lane']
line_names = ['road_divider', 'lane_divider']

for sequence in nuscenes_sequences:
    sequence_length=sequence['nbr_samples']
    frames_can_use=sequence_length-(seconds*2) #remove the last 4 seconds, that are 8 frames
    current_frame=0
    token  = sequence['first_sample_token']
    while(current_frame < frames_can_use):
        my_selected_sample = nuscenes.get('sample', token)
        tokens=get_following_tokens(seconds, token, my_selected_sample)
        
        pklfile_original, info_original, all_pkls_original, gtdist,preddist_original,gtxs, predxs_original, \
        createdimages_original, gt_boxes, nusc_maps=dt.calc_sample_crit_GOAL3_visualization(listtoken=tokens,
                                     conf_th=basic_threshold,
                                     crit=basic_criticality,
                                     correction_factor=None, #in this case, it is just the normal pkl
                                     obj_classes_list=object_classes,#filter boxes based on class
                                     image_counter=len(tokens),
                                     verbose=verbose);

        pklfile_dynamic, info_dynamic, all_pkls_dynamic, gtdist,preddist_dynamic,gtxs, predxs_dynamic, \
        createdimages_dynamic, gt_boxes, nusc_maps=dt.calc_sample_crit_GOAL3_visualization(listtoken=tokens,
                                     conf_th=basic_threshold,
                                     crit=basic_criticality,
                                     correction_factor=analyzed_correction_factor, #in this case, it is just the normal pkl
                                     obj_classes_list=object_classes,#filter boxes based on class
                                     image_counter=len(tokens),
                                     verbose=verbose);

        #for the trajectory with the original and dynamic approach
        do_computation(tokens,
                            info_original, createdimages_original, 
                            info_dynamic, createdimages_dynamic,
                            dx,bx, nx, ny,stretch, 
                            nuscenes, nusc_maps, gt_boxes )
        
        current_frame=current_frame+1;
        token=my_selected_sample['next']

In [ ]:
pp='/home/notebook/pkl/results/GOAL3/SSN/'
tken='62cc9dae3ca2420fa3629271cdd4212b_GT BBoxes and GT trajectory.npy'
a=np.load(pp+tken)

In [ ]:
%matplotlib inline
plt.figure()
plt.imshow(a)
plt.show()